In [1]:
import pandas as pd
import holidays
import joblib 
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score

In [4]:
data = pd.read_csv('hotel_booking.csv')


In [5]:
data.head()
data.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'name', 'email',
       'phone-number', 'credit_card'],
      dtype='object')

In [6]:
selected_columns = ['arrival_date_year','arrival_date_day_of_month','arrival_date_month', 'stays_in_weekend_nights','stays_in_week_nights', 'reserved_room_type']

In [7]:
df_new = data[selected_columns]

In [8]:
df_new.head()
df_new.shape


(119390, 6)

In [9]:
df_new.tail()

,arrival_date_year,arrival_date_day_of_month,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,reserved_room_type
119385,2017,30,August,2,5,A
119386,2017,31,August,2,5,E
119387,2017,31,August,2,5,D
119388,2017,31,August,2,5,A
119389,2017,29,August,2,7,A


In [10]:
df_new['stays_in_weekend_nights'].dtype

dtype('int64')

In [11]:
df_new['stays_in_weekend_nights'] = (df_new['stays_in_weekend_nights']>0).astype(int)

/var/folders/96/ljvyzg195d31021176thkm0c0000gn/T/ipykernel_46863/558426904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['stays_in_weekend_nights'] = (df_new['stays_in_weekend_nights']>0).astype(int)


In [12]:
df_new['stays_in_week_nights'] = (df_new['stays_in_week_nights']>0).astype(int)

/var/folders/96/ljvyzg195d31021176thkm0c0000gn/T/ipykernel_46863/1573806606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['stays_in_week_nights'] = (df_new['stays_in_week_nights']>0).astype(int)


In [13]:
df_new.head()

,arrival_date_year,arrival_date_day_of_month,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,reserved_room_type
0,2015,1,July,0,0,C
1,2015,1,July,0,0,C
2,2015,1,July,0,1,A
3,2015,1,July,0,1,A
4,2015,1,July,0,1,A


In [14]:
df_new = df_new.rename(columns={'arrival_date_year':'year', 'arrival_date_day_of_month':'day', 'arrival_date_month':'month', 'stays_in_weekend_nights':'weekend_nights', 'stays_in_week_nights':'week_nights', 'reserved_room_type':'room_type'})
df_new.head()

,year,day,month,weekend_nights,week_nights,room_type
0,2015,1,July,0,0,C
1,2015,1,July,0,0,C
2,2015,1,July,0,1,A
3,2015,1,July,0,1,A
4,2015,1,July,0,1,A


In [15]:
df_new.to_csv('New_hb.csv', index=False)


In [16]:
# A-0
df_new.to_csv('New_hb2.csv',index = False)

In [17]:
df_new = pd.read_csv('New_hb.csv')

In [18]:
df_new['month'] = pd.to_datetime(df_new['month'], format='%B').dt.month

# Create Date column
df_new['Date'] = pd.to_datetime(df_new[['year', 'month', 'day']])

# Define the country holidays (India holidays)
india_holidays = holidays.India()

# Add is_Holiday column (1 if holiday, 0 otherwise)
df_new['is_Holiday'] = df_new['Date'].apply(lambda x: 1 if x in india_holidays else 0)

# Drop the Date column if you no longer need it
df_new.drop(columns=['Date'], inplace=True)

# Save the updated dataset
# df_new.to_csv('updated_hotel_bookings.csv', index=False)

print(df_new.head())

   year  day  month  weekend_nights  week_nights room_type  is_Holiday
0  2015    1      7               0            0         C           0
1  2015    1      7               0            0         C           0
2  2015    1      7               0            1         A           0
3  2015    1      7               0            1         A           0
4  2015    1      7               0            1         A           0


In [19]:
encoder = LabelEncoder()
df_new["room_type"] = encoder.fit_transform(df_new["room_type"])

In [20]:
df_new.tail()

,year,day,month,weekend_nights,week_nights,room_type,is_Holiday
119385,2017,30,8,1,1,0,0
119386,2017,31,8,1,1,4,0
119387,2017,31,8,1,1,3,0
119388,2017,31,8,1,1,0,0
119389,2017,29,8,1,1,0,0


In [2]:
df_new.to_csv('updated_hotel_bookings.csv', index=False)

NameError: name 'df_new' is not defined

In [13]:
room_prices = {
    0: 2499,  
    1: 3999, 
    2: 5599,  
    3: 7499,  
    4: 14599
}

In [4]:
df_new = pd.read_csv('updated_hotel_bookings_price.csv')

In [23]:
def calculate_adr(row):
    base_price = room_prices.get(row["room_type"], 100)  # Default price if room_type is missing
    
    # Weekend premium (+20 per night)
    weekend_premium = row["weekend_nights"] * 120
    
    # Holiday surge pricing (+50)
    holiday_premium = 150 if row["is_Holiday"] == 1 else 0
    
    # Seasonal pricing adjustments
    if row["month"] in [4, 5, 6]:  # Peak summer season
        seasonal_premium = 130
    elif row["month"] in [12, 1,2]:  # Winter season
        seasonal_premium = 120
    else:
        seasonal_premium = 0
    
    # Calculate final ADR
    final_price = base_price + weekend_premium + holiday_premium + seasonal_premium
    return final_price


In [14]:
def calculate_adr(row):
    base_price = room_prices.get(row["room_type"], 100)  # Default price
    
    # Weekend premium (+120 per night)
    weekend_premium = row["weekend_nights"] * 399
    
    # Holiday surge pricing (+150)
    holiday_premium = 299 if row["is_Holiday"] == 1 else 0
    
    # Seasonal pricing adjustments
    if row["month"] in [4, 5, 6]:  # Peak summer season
        seasonal_premium = 599
    elif row["month"] in [12, 1, 2]:  # Winter season
        seasonal_premium = 599
    else:
        seasonal_premium = -399  # Off-season, apply a discount
    
    # Apply Discount for Low-Demand Periods
    discount = 999
    if seasonal_premium == 0 and holiday_premium == 0:
        discount = 1149  # Reduce base price by 50 if it's not peak season and not a holiday
        if row["weekend_nights"] == 0:  # Further decrease if no weekend nights
            discount += 749
    
    # **Date-based price factor**
    date_premium = 0
    if row["day"] in range(1, 8) or row["day"] in range(27, 32):
        date_premium = 299  # Increase price for bookings on 1-7 or 27-31
    
    # Calculate final ADR
    final_price = base_price + weekend_premium + holiday_premium + seasonal_premium + date_premium - discount
    
    return max(final_price, 2499)  # Ensure price doesn't drop too low

In [15]:
df_new = pd.read_csv('updated_hotel_bookings.csv')

In [16]:
df_new["adr"] = df_new.apply(calculate_adr, axis=1)

In [17]:
df_new.to_csv('updated_hotel_bookings_price_3.csv', index=False)

In [18]:
df_new = pd.read_csv('updated_hotel_bookings_price_3.csv')

In [19]:
X = df_new[["year", "day", "month", "weekend_nights", "week_nights", "room_type", "is_Holiday"]]
y = df_new["adr"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)


In [21]:
mae = mean_absolute_error(y_test, y_pred)
print(mae)

0.2182444090794869


In [22]:
r2 = r2_score(y_test, y_pred)

print(f"R² Score: {r2}")

R² Score: 0.9999955296971441


In [23]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

# Convert negative MAE to positive
cv_scores = -cv_scores  

print(f"Cross-validation MAE scores: {cv_scores}")
print(f"Mean MAE: {np.mean(cv_scores)}")

Cross-validation MAE scores: [86.95605591 87.15270829 87.66430665 86.55182559 85.60374029]
Mean MAE: 86.785727346924


In [24]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on training data
rf_model.fit(X_train, y_train)  # Ma

RandomForestRegressor(random_state=42)

In [25]:
joblib.dump(rf_model, 'random_forest_model_steve3.pkl')
print("Model saved successfully!")

Model saved successfully!


In [26]:
# Sample test input (replace with actual test data)
rf_model_loaded = joblib.load('random_forest_model_steve3.pkl')

In [27]:
test_input = np.array([[2025, 14, 3, 1, 0, 2, 1]])  # Example: [year, day, month, weekend_nights, week_nights, room_type, is_Holiday]
predicted_price = rf_model.predict(test_input)

print(f"Predicted Room Price: ${predicted_price[0]:.2f}")


Predicted Room Price: $4767.44


c:\Users\drums\anaconda3\envs\dpe\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
